In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.4 \n'

In [2]:
class histo_store:
    pass

fiducial_cut = False
data_mod_dirs=["sim", "sim-mod", "sim_2021"]
data_type = "_uncor_"
data_type_name = "UNCorrected"
if fiducial_cut:
    fiducial_cut_name = "fiducial"
else:
    fiducial_cut_name = ""

ss = []
for data_mod in data_mod_dirs:
    ss.append(histo_store())
    ss[-1].data_mod = data_mod
    ss[-1].ch = R.TChain("MiniDST")
    ss[-1].data_source="/data/HPS/data/physrun2021/" + data_mod
    ss[-1].energy="*"
    ss[-1].particle="e-"
    if ss[-1].particle == "e+":
        ss[-1].particle_s = "e\+"
    else:
        ss[-1].particle_s = ss[-1].particle
    if ss[-1].energy == "*":
        ss[-1].energy_s = "all_"
    else:
        ss[-1].energy_s = ss[-1].energy;
    ss[-1].ch.Add(ss[-1].data_source+"/hpsForward_"+ss[-1].particle_s+"_"+ss[-1].energy+"GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
    # ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
    print(f"Number of events loaded: {ss[-1].ch.GetEntries()/1e6:7.3f}M")
    ss[-1].df = R.RDataFrame(ss[-1].ch)
    ss[-1].dfx= ss[-1].df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
        .Filter("n_kf_track==1").Filter("ecal_cluster"+data_type+"energy.size()==1")
    if fiducial_cut:
        ss[-1].dfx= ss[-1].dfx.Define("is_fiducial","is_in_fiducial_region(ecal_cluster_seed_ix[0], ecal_cluster_seed_iy[0])") \
        .Filter("is_fiducial == true") \

    ss[-1].dfx= ss[-1].dfx \
        .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
        .Define("trk_gbl_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]>=32) return(i);} return(-1);") \
        .Define("diff_posx_ecal","ecal_cluster"+data_type+"x[0] - track_x_at_ecal[trk_idx]") \
        .Define("diff_posy_ecal","ecal_cluster"+data_type+"y[0] - track_y_at_ecal[trk_idx]") \
        .Define("diff_E_p","ecal_cluster"+data_type+"energy[0] - get_vec_abs(track_px,track_py,track_pz)[trk_idx]") \
        .Define("diff_p_kf_gbl","get_vec_abs(track_px,track_py,track_pz)[trk_idx] - get_vec_abs(track_px,track_py,track_pz)[trk_gbl_idx]") \
        .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
        .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")

    ss[-1].hx_diff_posx_ecal=ss[-1].dfx.Histo1D(("hx_diff_posx",data_type_name+" Diff Ecal_x - Track_x;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
    ss[-1].hx_diff_posx_ecal_top=ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_posx_top",data_type_name+" Diff Ecal_x - Track_x Top;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
    ss[-1].hx_diff_posx_ecal_bot=ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_posx_bot",data_type_name+" Diff Ecal_x - Track_x Bot;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
    ss[-1].hx_diff_posx_ecal_L=ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_posx_L",data_type_name+" Diff Ecal_x - Track_x L;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
    ss[-1].hx_diff_posx_ecal_R=ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_posx_R",data_type_name+" Diff Ecal_x - Track_x R;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
    ss[-1].hx_diff_posy_ecal=ss[-1].dfx.Histo1D(("hx_diff_posy",data_type_name+" Diff Ecal_y - Track_y;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
    ss[-1].hx_diff_posy_ecal_top=ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_posy_top",data_type_name+" Diff Ecal_y - Track_y Top;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
    ss[-1].hx_diff_posy_ecal_bot=ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_posy_bot",data_type_name+" Diff Ecal_y - Track_y Bot;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
    ss[-1].hx_diff_posy_ecal_L=ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_posy_L",data_type_name+" Diff Ecal_y - Track_y L;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
    ss[-1].hx_diff_posy_ecal_R=ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_posy_R",data_type_name+" Diff Ecal_y - Track_y R;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
    ss[-1].hx_diff_e_p = ss[-1].dfx.Histo1D(("hx_diff_e_p", data_type_name+" Diff Energy - Momentum;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
    ss[-1].hx_diff_e_p_top = ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_e_p_top", data_type_name+" Diff Energy - Momentum Top;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
    ss[-1].hx_diff_e_p_bot = ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_e_p_bot", data_type_name+" Diff Energy - Momentum Bot;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
    ss[-1].hx_diff_e_p_L = ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_e_p_L", data_type_name+" Diff Energy - Momentum Left;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
    ss[-1].hx_diff_e_p_R = ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_e_p_R",data_type_name+" Diff Energy - Momentum Right;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
    ss[-1].hx_diff_p_kf_gbl = ss[-1].dfx.Histo1D(("hx_diff_p_kf_gbl",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
    ss[-1].hx_diff_p_kf_gbl_top = ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_p_kf_gbl_top",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
    ss[-1].hx_diff_p_kf_gbl_bot = ss[-1].dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_p_kf_gbl_bot",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
    ss[-1].hx_diff_p_kf_gbl_L = ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_p_kf_gbl_L",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
    ss[-1].hx_diff_p_kf_gbl_R = ss[-1].dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_p_kf_gbl_R",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")

    ss[-1].hx_track_z_at_ecal = ss[-1].dfx.Histo1D(("hx_track_z_at_ecal","Track z at ECal; z[mm]", 301,1299.5,1500.5),"track_z_at_ecal")
    ss[-1].hx_ecal_cluster_z = ss[-1].dfx.Histo1D(("hx_ecal_cluster_z","Ecal cluster z position;z [mm]",301,1299.5,1500.5),"ecal_cluster_z")
    ss[-1].hx_ecal_cluster_uncor_z = ss[-1].dfx.Histo1D(("hx_ecal_cluster_z","Ecal cluster z position;z [mm]",301,1299.5,1500.5),"ecal_cluster_uncor_z")
    ss[-1].hx_mc_part_primary_z = ss[-1].dfx.Histo1D(("hx_mc_part_primary_z","End of track for primary MC particle.;z[mm]",1000,-0.1,1700.),"mc_part_primary_z")
    ss[-1].hx_mc_part_ave_daughter_z = ss[-1].dfx.Histo1D(("hx_mc_part_ave_daughter_z","Average end of track for primary MC particle daughters.;z[mm]",1000,-0.1,1700.),"mc_part_daughters_z")

Number of events loaded:  10.748M
Number of events loaded:  10.677M
Number of events loaded:  10.505M


In [3]:
%%time
h_colors = [R.kRed, R.kGreen, R.kBlue]
cc5a = R.TCanvas("cc5a","Canvas",800,600)
ss[0].hx_track_z_at_ecal.SetStats(0)
xax = ss[0].hx_track_z_at_ecal.GetXaxis()
xax.SetLimits(1380,1480)
ss[0].hx_track_z_at_ecal.Draw()
for i in range(len(ss)):
    ss[i].hx_track_z_at_ecal.SetLineColor(h_colors[i])
    ss[i].hx_track_z_at_ecal.SetFillColor(h_colors[i])
    ss[i].hx_track_z_at_ecal.Draw("same")
    ss[i].track_z_at_ecal_mean = ss[i].hx_track_z_at_ecal.GetMean()
cc5a.Draw()
#cc5a.SaveAs("track_z_at_ecal.pdf")

CPU times: user 6min 1s, sys: 12.8 s, total: 6min 13s
Wall time: 47.5 s


In [4]:
cc5b = R.TCanvas("cc5b","Canvas",800,600)
ss[0].hx_ecal_cluster_z.SetLineColor(R.kBlue+1)
ss[0].hx_ecal_cluster_z.SetLineWidth(2)
ss[0].hx_ecal_cluster_z.Draw()
#ss[0].hx_ecal_cluster_uncor_z.SetLineColor(R.kGreen+1)
#ss[0].hx_ecal_cluster_uncor_z.Draw()
cc5b.Draw()
cc5b.SaveAs("ecal_cluster_z.pdf")

Info in <TCanvas::Print>: pdf file ecal_cluster_z.pdf has been created


In [5]:
for s in ss:
    s.cc4 = R.TCanvas("cc4_"+s.data_mod,"Canvas",1200,600)
    s.cc4.Divide(2,2)
    s.p1=s.cc4.cd(1)
    #p1.SetLogy()
    s.hx_diff_posx_ecal.SetStats(0)
    s.hx_diff_posx_ecal.Draw()
    s.hx_diff_posx_ecal_top.SetLineColor(R.kGreen)
    s.hx_diff_posx_ecal_top.Draw("same")
    s.hx_diff_posx_ecal_bot.SetLineColor(R.kRed)
    s.hx_diff_posx_ecal_bot.Draw("same")
    s.hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
    s.hx_diff_posx_ecal_L.Draw("same")
    s.hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
    s.hx_diff_posx_ecal_R.Draw("same")
    s.p2=s.cc4.cd(2)
    #p2.SetLogy()
    s.hx_diff_posy_ecal.SetStats(0)
    s.hx_diff_posy_ecal.Draw()
    s.hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
    s.hx_diff_posy_ecal_top.Draw("same")
    s.hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
    s.hx_diff_posy_ecal_bot.Draw("same")
    s.hx_diff_posy_ecal_L.SetLineColor(R.kMagenta)
    s.hx_diff_posy_ecal_L.Draw("same")
    s.hx_diff_posy_ecal_R.SetLineColor(R.kCyan)
    s.hx_diff_posy_ecal_R.Draw("same")
    s.p3=s.cc4.cd(3)
    # p3.SetLogy()
    s.hx_diff_e_p.SetStats(0)
    s.hx_diff_e_p.Draw()
    s.hx_diff_e_p_top.SetLineColor(R.kGreen)
    s.hx_diff_e_p_top.Draw("same")
    s.hx_diff_e_p_bot.SetLineColor(R.kRed)
    s.hx_diff_e_p_bot.Draw("same")
    s.hx_diff_e_p_L.SetLineColor(R.kMagenta)
    s.hx_diff_e_p_L.Draw("same")
    s.hx_diff_e_p_R.SetLineColor(R.kCyan)
    s.hx_diff_e_p_R.Draw("same")
    s.p4=s.cc4.cd(4)
    # p4.SetLogy()
    s.hx_diff_p_kf_gbl.SetStats(0)
    s.hx_diff_p_kf_gbl.Draw()
    s.hx_diff_p_kf_gbl_top.SetLineColor(R.kGreen)
    s.hx_diff_p_kf_gbl_top.Draw("same")
    s.hx_diff_p_kf_gbl_bot.SetLineColor(R.kRed)
    s.hx_diff_p_kf_gbl_bot.Draw("same")
    s.hx_diff_p_kf_gbl_L.SetLineColor(R.kMagenta)
    s.hx_diff_p_kf_gbl_L.Draw("same")
    s.hx_diff_p_kf_gbl_R.SetLineColor(R.kCyan)
    s.hx_diff_p_kf_gbl_R.Draw("same")

    # h_trk_type.Draw()
    s.cc4.cd(0)
    s.leg1 = R.TLegend(0.8,0.8,0.94,0.94)
    s.leg1.AddEntry(s.hx_diff_posx_ecal.GetValue(), "All data with one cluster")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_top.GetValue(),"Top only")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
    s.leg1.Draw()

    s.cc4_text = R.TPaveText(0.43,0.93,0.57,0.99)
    s.cc4_text.AddText(f"Track endpoint = {s.track_z_at_ecal_mean:6.1f}")
    s.cc4_text.Draw()

    s.cc4.Draw()
    s.cc4.SaveAs(s.data_mod+"_"+s.particle+"_"+s.energy_s+"GeV"+data_type+fiducial_cut_name+"compare.pdf")

Info in <TCanvas::Print>: pdf file sim_e-_all_GeV_uncor_compare.pdf has been created
Info in <TCanvas::Print>: pdf file sim-mod_e-_all_GeV_uncor_compare.pdf has been created
Info in <TCanvas::Print>: pdf file sim-mod2_e-_all_GeV_uncor_compare.pdf has been created


In [6]:
# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tail(void){
auto gaustail = new TF1("gaustail",fit_gaus_tail,-20.,20.,4);
gaustail->SetParameters(1208, 0, 3.4, 1.);
gaustail->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustail function is now setup. " << std::endl;
}
''')
R.setup_gaus_tail()

# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tailn(void){
auto gaustailn = new TF1("gaustailn",fit_gaus_tailn,-20.,20.,4);
gaustailn->SetParameters(1208, 0, 3.4, 1.);
gaustailn->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustailn function is now setup. " << std::endl;
}
''')
R.setup_gaus_tailn()

The gaustail function is now setup. 
The gaustailn function is now setup. 


In [9]:
i =0;
for s in ss:
    s.cc6 = R.TCanvas("cc6_"+s.data_mod,"Canvas",1200,400)
    s.cc6.Divide(2,1)
    s.p1=s.cc6.cd(1)
#p1.SetLogy()
    s.hx_diff_posx_ecal_R.SetStats(0)
    s.hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
    s.ecal_right_diff_mean = s.hx_diff_posx_ecal_R.GetMean()
    R.gaustail.SetParameter(1,s.ecal_right_diff_mean)
    print("Fit results for the 'right' distribution (cyan)")
    s.fit_result_right = s.hx_diff_posx_ecal_R.Fit("gaus","S")
    s.hx_diff_posx_ecal_R.GetFunction("gaus").SetLineColor(R.kCyan+1);


    s.hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
    s.ecal_left_diff_mean = s.hx_diff_posx_ecal_L.GetMean()

    print("Fit results for the 'left' distribution (magenta) for "+s.data_mod)
    if i==0:
        s.fit_result_left = s.hx_diff_posx_ecal_L.Fit("gaustailn","S")
        s.hx_diff_posx_ecal_L.GetFunction("gaustailn").SetLineColor(R.kMagenta+1);
    else:
        R.gaustail.SetParameter(0, 12000)
        R.gaustail.SetParameter(1, 6.)
        R.gaustail.SetParameter(2, 2.8)
        R.gaustail.SetParameter(3, 0.01)
        s.fit_result_left = s.hx_diff_posx_ecal_L.Fit("gaustail","S")
        s.hx_diff_posx_ecal_L.GetFunction("gaustail").SetLineColor(R.kMagenta+1);


    s.hx_diff_posx_ecal_R.Draw()
    s.hx_diff_posx_ecal_L.Draw("same")


    s.p2=s.cc6.cd(2)
#p2.SetLogy()

    s.tophist = s.hx_diff_posy_ecal_top.GetPtr()
    s.xax = s.tophist.GetXaxis()
    s.xax.SetLimits(-10,10)
    s.hx_diff_posy_ecal_top.SetStats(0)
    s.hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
    s.ecal_top_diff_mean = s.hx_diff_posy_ecal_top.GetMean()
    R.gaustail.SetParameter(1,s.ecal_top_diff_mean)
    print("Fit results for the 'top' distribution (green) for "+s.data_mod)
    s.fit_result_top = s.hx_diff_posy_ecal_top.Fit("gaus","S")
    s.hx_diff_posy_ecal_top.GetFunction("gaus").SetLineColor(R.kGreen+1);


    s.hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
    s.ecal_bot_diff_mean = s.hx_diff_posy_ecal_bot.GetMean()
    R.gaustail.SetParameter(1,s.ecal_bot_diff_mean)
    print("Fit results for the 'bottom' distribution (red) for "+s.data_mod)
    s.fit_result_bot = s.hx_diff_posy_ecal_bot.Fit("gaus","S")
    s.hx_diff_posy_ecal_bot.GetFunction("gaus").SetLineColor(R.kRed+1);

    s.hx_diff_posy_ecal_top.Draw()
    s.hx_diff_posy_ecal_bot.Draw("same")


# h_trk_type.Draw()
    s.cc6.cd(0)
    s.leg1 = R.TLegend(0.84,0.76,0.94,0.9)
    s.leg1.AddEntry(s.hx_diff_posx_ecal.GetValue(), "All data with one cluster")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_top.GetValue(),"Top only")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
    s.leg1.AddEntry(s.hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
    s.leg1.Draw()

    s.cc6.cd(0)
    # s.cc4_text = R.TPaveText(0.43,0.93,0.57,0.99)
    # s.cc4_text.AddText(f"Track endpoint = {s.track_z_at_ecal_mean:6.1f}")
    s.cc4_text.Draw()

    s.cc6.Draw()
    s.cc6.SaveAs("LR_top_bottom_fits_"+s.data_mod+data_type+fiducial_cut_name+".pdf")
    i += 1

Fit results for the 'right' distribution (cyan)
Fit results for the 'left' distribution (magenta) for sim
Fit results for the 'top' distribution (green) for sim
Fit results for the 'bottom' distribution (red) for sim
Fit results for the 'right' distribution (cyan)
Fit results for the 'left' distribution (magenta) for sim-mod
Fit results for the 'top' distribution (green) for sim-mod
Fit results for the 'bottom' distribution (red) for sim-mod
Fit results for the 'right' distribution (cyan)
Fit results for the 'left' distribution (magenta) for sim-mod2
Fit results for the 'top' distribution (green) for sim-mod2
Fit results for the 'bottom' distribution (red) for sim-mod2
 FCN=23166.9 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=1.84312e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6_sim
Info in <TCanvas::Print>: pdf file LR_top_bottom_fits_sim_fiducial.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6_sim-mod
Info in <TCanvas::Print>: pdf file LR_top_bottom_fits_sim-mod_fiducial.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6_sim-mod2
Info in <TCanvas::Print>: pdf file LR_top_bottom_fits_sim-mod2_fiducial.pdf has been created


In [10]:
for s in ss:
    print("Fit results for the 'top' distribution (green) for "+s.data_mod)
    print(f"Simple mean = {s.ecal_top_diff_mean}")
    print(f"Fit mean    = {s.fit_result_top.Parameter(1)} ± {s.fit_result_top.Error(1)}   Chi2/Ndf = {s.fit_result_top.Chi2()/s.fit_result_top.Ndf()}")
    # print(s.fit_result_top)
print("")
print("--------------------------------------------------------------------------------------------------")

for s in ss:
    print("Fit results for the 'bottom' distribution (red) for "+s.data_mod)
    print(f"Simple mean = {s.ecal_bot_diff_mean}")
    print(f"Fit mean    = {s.fit_result_bot.Parameter(1)} ± {s.fit_result_bot.Error(1)}   Chi2/Ndf = {s.fit_result_bot.Chi2()/s.fit_result_bot.Ndf()}")
    # print(s.fit_result_bot)

    print("")
print("--------------------------------------------------------------------------------------------------")
    
for s in ss:
    print("Fit results for the 'Left' distribution (Magenta) for "+s.data_mod)
    print(f"Simple mean = {s.ecal_left_diff_mean}")
    print(f"Fit mean    = {s.fit_result_left.Parameter(1)} ± {s.fit_result_left.Error(1)}   Chi2/Ndf = {s.fit_result_left.Chi2()/s.fit_result_left.Ndf()}")
    # print(s.fit_result_top)
print("")
print("--------------------------------------------------------------------------------------------------")
for s in ss:
    print("Fit results for the 'Right' distribution (Cyan) for "+s.data_mod)
    print(f"Simple mean = {s.ecal_right_diff_mean}")
    print(f"Fit mean    = {s.fit_result_right.Parameter(1)} ± {s.fit_result_right.Error(1)}   Chi2/Ndf = {s.fit_result_right.Chi2()/s.fit_result_right.Ndf()}")
    # print(s.fit_result_bot)

Fit results for the 'top' distribution (green) for sim
Simple mean = 1.680025775580353
Fit mean    = 0.858634978923269 ± 0.0005058036514878983   Chi2/Ndf = 26.63395860986805
Fit results for the 'top' distribution (green) for sim-mod
Simple mean = -1.3206823268898815
Fit mean    = -0.6487669821144975 ± 0.0005201392077620743   Chi2/Ndf = 23.088661664671072
Fit results for the 'top' distribution (green) for sim-mod2
Simple mean = -0.34754358800261437
Fit mean    = -0.15918698380405233 ± 0.0005123501860791255   Chi2/Ndf = 25.63682163003096

--------------------------------------------------------------------------------------------------
Fit results for the 'bottom' distribution (red) for sim
Simple mean = -0.6544547958560578
Fit mean    = -0.689192372099799 ± 0.0010215818696820177   Chi2/Ndf = 26.410741967497845

Fit results for the 'bottom' distribution (red) for sim-mod
Simple mean = 2.2981570009687196
Fit mean    = 2.277907040631294 ± 0.0010528782939708873   Chi2/Ndf = 22.4076497259141

In [11]:
location_z = []
top_mean = []
top_mean_e = []
bot_mean = []
bot_mean_e = []
left_mean = []
left_mean_e = []
right_mean = []
right_mean_e = []
for s in ss:
    location_z.append(s.track_z_at_ecal_mean)
    top_mean.append(s.fit_result_top.Parameter(1))
    top_mean_e.append(s.fit_result_top.Error(1))
    bot_mean.append(s.fit_result_bot.Parameter(1))
    bot_mean_e.append(s.fit_result_bot.Error(1))
    left_mean.append(s.fit_result_left.Parameter(1))
    left_mean_e.append(s.fit_result_left.Error(1))
    right_mean.append(s.fit_result_right.Parameter(1))
    right_mean_e.append(s.fit_result_right.Error(1))

In [12]:
from numpy.polynomial import Polynomial
pfit_top = Polynomial.fit(location_z, top_mean, 1)
top_dat = pfit_top.linspace(2)
pfit_bot = Polynomial.fit(location_z, bot_mean, 1)
bot_dat = pfit_bot.linspace(2)
pfit_left = Polynomial.fit(location_z, left_mean, 1)
left_dat = pfit_left.linspace(2)
pfit_right = Polynomial.fit(location_z, right_mean, 1)
right_dat = pfit_right.linspace(2)

In [13]:
cc7 = R.TCanvas("cc7","CC7", 800,600)
h_empty = R.TH2F("h_empty","Crossing of track paths",100,1384,1484,100,-5., 6)
h_empty.SetStats(0)
h_empty.Draw()

top_markers=[]
bot_markers=[]
left_markers=[]
right_markers=[]

for i in range(len(top_mean)):
    top_markers.append(R.TMarker(location_z[i], top_mean[i],21))
    top_markers[-1].SetMarkerColor(R.kGreen+1)
    top_markers[-1].Draw()
    bot_markers.append(R.TMarker(location_z[i], bot_mean[i],21))
    bot_markers[-1].SetMarkerColor(R.kRed+1)
    bot_markers[-1].Draw()
    left_markers.append(R.TMarker(location_z[i], left_mean[i],21))
    left_markers[-1].SetMarkerColor(R.kMagenta+1)
    left_markers[-1].Draw()
    right_markers.append(R.TMarker(location_z[i], right_mean[i],21))
    right_markers[-1].SetMarkerColor(R.kCyan+1)
    right_markers[-1].Draw()

l_top = R.TLine(top_dat[0][0], top_dat[1][0], top_dat[0][1], top_dat[1][1])
l_top.SetLineColor(R.kGreen)
l_top.SetLineWidth(2)
l_top.Draw()
l_bot = R.TLine(bot_dat[0][0], bot_dat[1][0], bot_dat[0][1], bot_dat[1][1])
l_bot.SetLineColor(R.kRed)
l_bot.SetLineWidth(2)
l_bot.Draw()
l_left = R.TLine(left_dat[0][0], left_dat[1][0], left_dat[0][1], left_dat[1][1])
l_left.SetLineColor(R.kMagenta)
l_left.SetLineWidth(2)
l_left.Draw()
l_right = R.TLine(right_dat[0][0], right_dat[1][0], right_dat[0][1], right_dat[1][1])
l_right.SetLineColor(R.kCyan)
l_right.SetLineWidth(2)
l_right.Draw()
l_zero = R.TLine(1384, 0, 1484, 0)
l_zero.SetLineColor(R.kBlack)
l_zero.Draw()
cc7.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(l_top,"Top")
leg1.AddEntry(l_bot,"Bottom")
leg1.AddEntry(l_left,"Left x<0")
leg1.AddEntry(l_right,"Right x>0")
leg1.Draw()

cc7.Draw()
cc7.SaveAs("Fit_of_fits"+data_type+fiducial_cut_name+".pdf")

Info in <TCanvas::Print>: pdf file Fit_of_fits_fiducial.pdf has been created


# Compare ECal with the Scoring Plane

In [14]:
s = ss[2]
s.dfxx = s.dfx.Define("score_energy","score_plane_hit_energy(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
    .Define("score_x","score_plane_hit_x(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
    .Define("score_y","score_plane_hit_y(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
    .Define("n_score","score_energy.size()") \
    .Filter("n_score == 1") \
    .Define("score_ecal_diff_x","score_x[0] - ecal_cluster"+data_type+"x[0]") \
    .Define("score_ecal_diff_y","score_y[0] - ecal_cluster"+data_type+"y[0]") \
    .Define("score_ecal_diff_energy","score_energy[0] - ecal_cluster"+data_type+"energy[0]") \
    .Define("score_track_diff_x","score_x[0] - track_x_at_ecal[trk_idx]") \
    .Define("score_track_diff_y","score_y[0] - track_y_at_ecal[trk_idx]")

In [15]:
s.hxx_diff_score_ecal_posx = s.dfxx.Histo1D(("hxx_diff_score_ecal_posx","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
s.hxx_diff_score_ecal_posx_top = s.dfxx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hxx_diff_score_ecal_posx_top","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
s.hxx_diff_score_ecal_posx_bot = s.dfxx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hxx_diff_score_ecal_posx_bot","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
s.hxx_diff_score_ecal_posx_L = s.dfxx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hxx_diff_score_ecal_posx_L","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
s.hxx_diff_score_ecal_posx_R = s.dfxx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hxx_diff_score_ecal_posx_R","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")

s.hxx_diff_score_ecal_posy = s.dfxx.Histo1D(("hxx_diff_score_ecal_posy","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
s.hxx_diff_score_ecal_posy_top = s.dfxx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hxx_diff_score_ecal_posy_top","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
s.hxx_diff_score_ecal_posy_bot = s.dfxx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hxx_diff_score_ecal_posy_bot","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
s.hxx_diff_score_ecal_posy_L = s.dfxx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hxx_diff_score_ecal_posy_L","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
s.hxx_diff_score_ecal_posy_R = s.dfxx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hxx_diff_score_ecal_posy_R","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")



s.hxx_diff_score_track_posx = s.dfxx.Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
s.hxx_diff_score_track_posx_top = s.dfxx.Filter("track_y_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
s.hxx_diff_score_track_posx_bot = s.dfxx.Filter("track_y_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
s.hxx_diff_score_track_posx_L = s.dfxx.Filter("track_x_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
s.hxx_diff_score_track_posx_R = s.dfxx.Filter("track_x_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")

s.hxx_diff_score_track_posy = s.dfxx.Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
s.hxx_diff_score_track_posy_top = s.dfxx.Filter("track_y_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
s.hxx_diff_score_track_posy_bot = s.dfxx.Filter("track_y_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
s.hxx_diff_score_track_posy_L = s.dfxx.Filter("track_x_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
s.hxx_diff_score_track_posy_R = s.dfxx.Filter("track_x_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")

In [17]:
cc8 = R.TCanvas("cc8","CC8", 1200, 400)
cc8.Divide(2,1)

p1 = cc8.cd(1)
s.hxx_diff_score_ecal_posx.SetStats(0)
s.hxx_diff_score_ecal_posx.Draw()
s.hxx_diff_score_ecal_posx_top.SetLineColor(R.kGreen)
s.hxx_diff_score_ecal_posx_top.Draw("same")
s.hxx_diff_score_ecal_posx_bot.SetLineColor(R.kRed)
s.hxx_diff_score_ecal_posx_bot.Draw("same")
s.hxx_diff_score_ecal_posx_L.SetLineColor(R.kMagenta)
s.hxx_diff_score_ecal_posx_L.Draw("same")
s.hxx_diff_score_ecal_posx_R.SetLineColor(R.kCyan)
s.hxx_diff_score_ecal_posx_R.Draw("same")

p2 = cc8.cd(2)
s.hxx_diff_score_ecal_posy.SetStats(0)
s.hxx_diff_score_ecal_posy.Draw()
s.hxx_diff_score_ecal_posy_top.SetLineColor(R.kGreen)
s.hxx_diff_score_ecal_posy_top.Draw("same")
s.hxx_diff_score_ecal_posy_bot.SetLineColor(R.kRed)
s.hxx_diff_score_ecal_posy_bot.Draw("same")
s.hxx_diff_score_ecal_posy_L.SetLineColor(R.kMagenta)
s.hxx_diff_score_ecal_posy_L.Draw("same")
s.hxx_diff_score_ecal_posy_R.SetLineColor(R.kCyan)
s.hxx_diff_score_ecal_posy_R.Draw("same")

cc8.cd(0)
s.leg2 = R.TLegend(0.8,0.75,0.94,0.9)
s.leg2.AddEntry(s.hxx_diff_score_ecal_posx.GetValue(), "All data with one cluster")
s.leg2.AddEntry(s.hxx_diff_score_ecal_posx_top.GetValue(),"Top only")
s.leg2.AddEntry(s.hxx_diff_score_ecal_posx_bot.GetValue(),"Bottom only")
s.leg2.AddEntry(s.hxx_diff_score_ecal_posx_L.GetValue(),"Left Only x<0")
s.leg2.AddEntry(s.hxx_diff_score_ecal_posx_R.GetValue(),"Right Only x>0")
s.leg2.Draw()

cc8.Draw()
cc8.SaveAs(s.data_mod+"_"+s.particle+"_"+s.energy_s+"GeV"+data_type+fiducial_cut_name+"score_ecal_compare.pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc8
Info in <TCanvas::Print>: pdf file sim-mod2_e-_all_GeV_fiducialscore_ecal_compare.pdf has been created


In [18]:
cc9 = R.TCanvas("cc9","CC9", 1200, 400)
cc9.Divide(2,1)
p1 = cc9.cd(1)
s.hxx_diff_score_track_posx.SetStats(0)
s.hxx_diff_score_track_posx.Draw()
s.hxx_diff_score_track_posx_top.SetLineColor(R.kGreen)
s.hxx_diff_score_track_posx_top.Draw("same")
s.hxx_diff_score_track_posx_bot.SetLineColor(R.kRed)
s.hxx_diff_score_track_posx_bot.Draw("same")
s.hxx_diff_score_track_posx_L.SetLineColor(R.kMagenta)
s.hxx_diff_score_track_posx_L.Draw("same")
s.hxx_diff_score_track_posx_R.SetLineColor(R.kCyan)
s.hxx_diff_score_track_posx_R.Draw("same")

p2 = cc9.cd(2)
s.hxx_diff_score_track_posy.SetStats(0)
s.hxx_diff_score_track_posy.Draw()
s.hxx_diff_score_track_posy_top.SetLineColor(R.kGreen)
s.hxx_diff_score_track_posy_top.Draw("same")
s.hxx_diff_score_track_posy_bot.SetLineColor(R.kRed)
s.hxx_diff_score_track_posy_bot.Draw("same")
s.hxx_diff_score_track_posy_L.SetLineColor(R.kMagenta)
s.hxx_diff_score_track_posy_L.Draw("same")
s.hxx_diff_score_track_posy_R.SetLineColor(R.kCyan)
s.hxx_diff_score_track_posy_R.Draw("same")

cc9.cd(0)
s.leg2.Draw()

cc9.Draw()
cc9.SaveAs(s.data_mod+"_"+s.particle+"_"+s.energy_s+"GeV"+data_type+fiducial_cut_name+"score_track_compare.pdf")

Info in <TCanvas::Print>: pdf file sim-mod2_e-_all_GeV_fiducialscore_track_compare.pdf has been created


Info in <TCanvas::Print>: pdf file sim-mod2_e-_all_GeV_fiducialscore_track_compare.pdf has been created
